In [1]:
import numpy as np
import pandas as pd
import os, gc
from glob import glob
from tqdm import tqdm

import sys
sys.path.append(f'/home/{os.environ.get("USER")}/PythonLibrary')
import lgbextension as ex
import lightgbm as lgb
from multiprocessing import cpu_count

In [30]:
X_train = pd.read_csv('../input/train.csv.zip')
# X_train['ind'] = X_train.index

In [31]:
y_train = X_train['target']
X_train = X_train.iloc[:,2:]

In [32]:
X_train_0 = X_train[y_train==0]
X_train_1 = X_train[y_train==1]

In [33]:
def shuffle(df):
    df_ = pd.DataFrame(index=df.index)
    for c in tqdm(df.columns):
        df_[c] = df[c].sample(frac=1).reset_index(drop=True)
    return df_


In [34]:
X_train_0_ = shuffle(X_train_0)

100%|██████████| 200/200 [00:03<00:00, 51.46it/s]


In [35]:
X_train_1_ = shuffle(X_train_1)

100%|██████████| 200/200 [00:00<00:00, 366.96it/s]


In [36]:
NFOLD = 5

LOOP = 1

param = {
         'objective': 'binary',
         'metric': 'None',
         
         'learning_rate': 0.1,
         'max_depth': -1,
         'num_leaves': 2**6 -1,
         'max_bin': 255,
         
         'min_child_weight': 10,
         'min_data_in_leaf': 150,
         'reg_lambda': 0.5,  # L2 regularization term on weights.
         'reg_alpha': 0.5,  # L1 regularization term on weights.
         
         'colsample_bytree': 0.5,
         'subsample': 0.7,
#         'nthread': 32,
         'nthread': cpu_count(),
         'bagging_freq': 1,
         'verbose':-1,
         }


NROUND = 9999
ESR = 100
VERBOSE_EVAL = 50
SEED = 1

In [37]:
X_train_ = pd.concat([X_train_0, X_train_1, X_train_0_, X_train_1_]).sort_index()
y_train_ = pd.concat([y_train, y_train]).sort_index()

In [38]:
dtrain = lgb.Dataset(X_train_, y_train_.values, 
                     free_raw_data=False)
gc.collect()

757

In [39]:
from sklearn.model_selection import GroupKFold
group_kfold = GroupKFold(n_splits=NFOLD)


In [40]:
group = X_train_.index % NFOLD

In [41]:
X_train_0.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [42]:
X_train_0_.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,9.8057,1.1484,9.3518,5.8374,8.9834,-6.0953,4.8313,19.9431,3.1480,7.7871,...,-0.5557,5.1993,0.7529,2.7216,15.2105,0.0826,7.7853,9.1738,8.4487,-17.4818
1,9.4723,-1.2362,11.2790,5.6657,9.2472,-10.3247,5.5449,14.5063,2.0321,8.5359,...,2.5078,5.9663,3.3598,2.1598,16.2838,-0.9314,-2.3164,9.7627,19.5065,-4.7064
2,14.7137,-1.1544,6.1803,8.8005,9.4156,-4.4229,6.0883,16.8747,0.1621,9.8454,...,-1.5841,1.3081,0.8728,5.5021,11.6783,1.4207,11.2343,8.4743,16.9761,-15.5325
3,10.6806,1.4894,12.5201,5.6291,10.9236,0.1030,4.6909,17.8188,-0.7267,6.6151,...,7.4697,7.1580,1.3351,0.1119,21.3292,0.5884,9.3844,9.6075,16.4854,-2.9191
4,14.0891,-0.7006,13.7848,9.7052,10.9581,-18.7594,6.6074,18.2407,0.1286,7.9766,...,8.4440,8.0335,2.0963,5.6132,20.3021,-0.8228,-5.2073,8.4264,12.9990,2.1093


In [44]:
model_all = []
nround_mean = 0
loss_list = []
y_preds = []
for i in range(LOOP):
    gc.collect()
    
    param['seed'] = np.random.randint(9999)
    
    ret, models = lgb.cv(param, dtrain, NROUND,
#                          nfold=NFOLD,
                         folds=group_kfold.split(X_train_, y_train_, group),
                         stratified=True, shuffle=True,
                         feval=ex.eval_auc,
                         early_stopping_rounds=ESR, 
                         verbose_eval=VERBOSE_EVAL,
                         seed=SEED+i)
    
    y_pred = ex.eval_oob(X_train_, y_train_.values, models, SEED+i, 
                         folds=group_kfold.split(X_train_, y_train_, group),
                         stratified=True, shuffle=True)
    y_preds.append(y_pred)
    
    model_all += models
    nround_mean += len(ret['auc-mean'])
    loss_list.append( ret['auc-mean'][-1] )

nround_mean = int((nround_mean/LOOP) * 1.3)


[50]	cv_agg's auc: 0.883568 + 0.00118499
[100]	cv_agg's auc: 0.89747 + 0.000613525
[150]	cv_agg's auc: 0.903823 + 0.000620377
[200]	cv_agg's auc: 0.907004 + 0.0009314
[250]	cv_agg's auc: 0.90795 + 0.000873459
[300]	cv_agg's auc: 0.908571 + 0.00124696
[350]	cv_agg's auc: 0.909613 + 0.00196868
[400]	cv_agg's auc: 0.90977 + 0.000718653


In [45]:
test = pd.read_csv('../input/test.csv.zip')

In [47]:
sub = pd.read_csv('../input/sample_submission.csv.zip')

In [49]:
for model in tqdm(models):
    sub['target'] += pd.Series(model.predict(test.iloc[:,1:])).rank()
sub['target'] /= sub['target'].max()

100%|██████████| 5/5 [00:20<00:00,  4.09s/it]


In [50]:
sub

,ID_code,target
0,test_0,0.755767
1,test_1,0.893715
2,test_2,0.862423
3,test_3,0.843218
4,test_4,0.587445
5,test_5,0.012886
6,test_6,0.169668
7,test_7,0.823456
8,test_8,0.039915
9,test_9,0.156455
